In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [3]:
pff = pd.read_csv('pff.csv', index_col=0)
pff.head(2)

,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PF,PA,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
0,Arizona Cardinals,San Francisco 49Ers,2006,1,09/10/2006,4:15pm,H,W,34,27,72.5,72.6,75.5,72.4,76.5,47.4,65.6,67.7,69.8,37.5,56.2,70.7,-,63.8,64.7,54.5,83.6,62.0,67.4,63.6,61.4,63.2,32.0,69.0,53.8,-
1,Arizona Cardinals,Seattle Seahawks,2006,2,09/17/2006,4:05pm,A,L,10,21,48.0,50.1,27.9,57.1,65.3,59.7,62.1,49.8,38.3,44.7,60.5,62.3,-,80.2,70.2,56.5,68.6,57.6,63.5,85.4,85.5,72.8,61.2,73.4,84.0,-


In [9]:
weeks = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']

In [11]:
pff = pff[pff['Week'].isin(weeks)]

In [13]:
pff = pff.sort_values(by=['Season','Date','Time','Team']).reset_index().drop(columns='index')

In [14]:
pff.to_csv('pff_pi.csv')